In [1]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

In [2]:
DATA_DIR = "data/stanford_dogs/Images"
OUTPUT_DIR = "data/Processed"
IMG_SIZE = (224, 224)

In [3]:
def load_dataset(data_dir):
    all_paths = glob(os.path.join(data_dir, "*", "*.jpg"))
    all_labels = [os.path.basename(os.path.dirname(p)) for p in all_paths]
    return all_paths, all_labels

image_paths, labels = load_dataset(DATA_DIR)
print(f"Found {len(image_paths)} images")

Found 20580 images


In [4]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

for path, label in tqdm(zip(image_paths, labels), total=len(image_paths)):
    image = cv2.imread(path)
    if image is None:
        continue
    
    # image = resize_image(image)
    # image = normalize_image(image)
    
    save_path = os.path.join(OUTPUT_DIR, label)
    os.makedirs(save_path, exist_ok=True)
    filename = os.path.basename(path)
    
    save_img = (np.clip(image, 0, 1) * 255).astype(np.uint8)
    cv2.imwrite(os.path.join(save_path, filename), save_img)

  0%|          | 0/20580 [00:00<?, ?it/s]